In [1]:
from dotenv import load_dotenv


from langchain_groq import ChatGroq


from langgraph.graph import StateGraph , START , END
from langgraph.prebuilt import ToolNode , tools_condition

from typing_extensions import TypedDict
from langgraph.graph.message import add_messages , Annotated 
from langchain.messages import HumanMessage , AIMessage , AnyMessage

import os

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
llm = ChatGroq(model = "openai/gpt-oss-20b")

In [5]:
class State(TypedDict) :
     
    topic : str
    story : str 
    improved_story : str
    final_story : str


In [17]:
def generate(state : State) :
    response = llm.invoke(f"Write a one sentence story about the topic : {state['topic']} ")
    return {"story" : response.content}


def improve(state : State) :
    response = llm.invoke(f"Improve the story : {state['story']}")
    return{"improved_story" : response.content}


def polish(state : State) :
    response = llm.invoke(f" Give the final story with more vivid details: {state['improved_story']}")
    return{"final_story" : response.content}

In [18]:
def check_conflict(state : State) :

    if '?' in state['story'] or '!' in state['story'] :
        return "Fail"
    else :
        return "Pass"

In [19]:
graph = StateGraph(State)

In [20]:
# Add nodes
graph.add_node("generate" , generate)
graph.add_node("improve" , improve)
graph.add_node("polish" , polish)

In [21]:
graph.add_edge(START , "generate")

graph.add_conditional_edges("generate" , check_conflict, {"Pass" : "improve" , "Fail" : "generate" })

graph.add_edge("improve", "polish")
graph.add_edge("polish" , END)


graph_builder = graph.compile()

In [22]:
graph_builder.invoke({"topic" : "Space Exploration" })

{'topic': 'Space Exploration',
 'story': 'When the first probe finally touched the moon’s dust, it left a single, glowing footprint that would later become the blueprint for humanity’s dream to plant a garden on Mars.',
 'improved_story': 'When the first probe finally kissed the moon’s silvery dust, its tiny wheels left a single, luminous trail—an almost imperceptible halo that glimmered against the endless black. Engineers back on Earth stared at the live feed, watching that faint glow ripple across the cratered surface. It was a quiet miracle, a tiny promise that humanity could leave a mark beyond its own world.\n\nThat glowing footprint, recorded in every line of code and every pixel of the footage, became the seed from which a grander dream sprouted. Scientists began to imagine not just a robotic presence, but a living one—an ecosystem that could thrive on the barren plains of Mars. They drew up blueprints that borrowed the moon’s footprint as a template: a network of solar panels,